# Single-sample gene regulatory network inference with PANDA and LIONESS
This workflow generates single sample PANDA-LIONESS GRNs using the python implementations of the tools from The Network Zoo and can be used to reproduce the single sample networks used in (Pop, et al., 2024). As they can take a long time to run and are computationally demanding, we also provide the pre-computed indegrees and outdegrees used for the analysis in the Zenodo repository.

In this notebook, we use the files prepared in `single_sample_networks_data_prep.ipynb` to generate single sample networks. 

First, we load necessary libraries and set some global parameters.

In [ ]:
import os
from netZooPy.panda import Panda
from netZooPy.lioness import Lioness
from netZooPy.lioness.analyze_lioness import AnalyzeLioness
import pandas as pd

In [ ]:
# Set working directory
wd = "JDRnet"
os.chdir(wd)

In [ ]:

data_tcga = "data/TCGA"
data_gep = "data/GEPliver"

# Define a list of cancer names for which to generate networks
cancers_tcga = ["aml", "breast", "colon", "gbm", "kidney", "liver", "lung",
                "melanoma", "ovarian", "sarcoma"]

In [ ]:
# Set working directory
wd = "JDRnet"
os.chdir(wd)

data_tcga = "/data/TCGA"
data_gep = "/data/GEPliver"

# Define a list of cancer names for which to generate networks
cancers_tcga = ["aml", "breast", "colon", "gbm", "kidney", "liver", "lung",
                "melanoma", "ovarian", "sarcoma"]

# Generating networks
We use the `Panda` function from `netZooPy` to create an aggregate network with PANDA. Then we use the Lioness` function to generate the sample specific networks. We do this first for the TCGA datasets, then for the GEPliver dataset.

In [ ]:
for cancer in cancers_tcga:
    ppath = os.path.join(data_tcga, cancer)
    panda_obj = Panda(ppath+'/log_exp_fil.txt',
                  ppath+'/prior.txt',
                  ppath+'/ppi.txt',
                  remove_missing=False, 
                  keep_expression_matrix=True, save_memory=False, modeProcess='legacy')
    lioness_obj = Lioness(panda_obj)
    
    #save the edgelists
    lioness_obj.save_fmt='txt'
    lioness_obj.save_lioness_results()

    # calculate indegree & outdegree
    nets = lioness_obj.export_lioness_results
    ind = nets.groupby(by="gene").sum().drop("tf", axis = 1)
    out = nets.groupby(by="tf").sum().drop("gene", axis = 1)

    #save
    ind.to_csv('indegree.txt', sep='\t', index=False)
    out.to_csv('outdegree.txt', sep='\t', index=False)


In [ ]:
ppath = os.path.join(data_gep, cancer)
panda_obj = Panda(ppath+'log_exp_fil.txt',
                ppath+'prior.txt',
                ppath+'ppi.txt',
                remove_missing=False, 
                keep_expression_matrix=True, save_memory=False, modeProcess='legacy')

#save the edgelists
lioness_obj.save_fmt='txt'
lioness_obj.save_lioness_results()

# calculate indegree & outdegree
nets = lioness_obj.export_lioness_results
ind = nets.groupby(by="gene").sum().drop("tf", axis = 1)
out = nets.groupby(by="tf").sum().drop("gene", axis = 1)

#save
ind.to_csv('indegree.txt', sep='\t', index=False)
out.to_csv('outdegree.txt', sep='\t', index=False)